<a href="https://colab.research.google.com/github/lenonborges/Optimizing-Restaurant-s-Search/blob/main/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing the search for restaurants 

## Summary
-Introduction

-Business Problem

-Data to solve the problem

-Results

-Discussion

-Conclusion

## Introduction
This project's goal is to deliver the n best food venues (considering the best quality x price relation) near you. We will get the data for some food venues places in Rio de Janeiro since the Foursquare API limits the response objects.

As human beings we tend to get annoyed of going to the same places over and over, so it would be nice if we got a list of the n best options in a price vs quality view.
Then we can choose amongst a list which best suit for us on that particular moment. 

## Business Problem 
Nowadays, on May 2021, Google maps allows filter for price and quality given a particular area on the map. It also gives you a list of places with filter for quality levels in steps of 0.5 points of difference. Although, Google maps already has great filters such as: Romantic, Cozy, Fun and others. However there is no filter for a good and not expensive place to go.  

Therefore, the audience of this project is every person who wants to get a new, good and affordable place to eat.

## Data to solve the problem
We'll use the Foursquare API to gather data for venues in a given City. The data is such as location, latitude, longitude, name, price, rating and others. 

With these data we will construct a factor given weights for price and rating, actually 30% for price and 70% for rating. Where price has a negative effect and rating has a positive effect on our factor.

We will also use the Follium library to plot that found venues on map. 







In [1]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests

In [3]:
ID = ''
SECRET = ''
cidade = 'Rio de Janeiro'
vers = '20210324'
oauth= ''
categoria = '4bf58dd8d48988d110941735' #italian
#diner '4bf58dd8d48988d147941735' #food '4d4b7105d754a06374d81259' #japones '4bf58dd8d48988d111941735' australian '4bf58dd8d48988d169941735'
#french '4bf58dd8d48988d10c941735'
#Tech Startup 4bf58dd8d48988d125941735
#italian 4bf58dd8d48988d110941735

In [4]:
url = 'https://api.foursquare.com/v2/venues/search/?near={}&categoryId={}&cliend_id={}&client_secret={}&oauth_token={}&v={}'.format(cidade, categoria, ID, SECRET,oauth, vers)
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '62e02589355f5315f9a81f84'},
 'notifications': [{'item': {'unreadCount': 0}, 'type': 'notificationTray'}],
 'response': {'confident': False,
  'geocode': {'feature': {'cc': 'BR',
    'displayName': 'Rio de Janeiro, RJ, Brazil',
    'geometry': {'bounds': {'ne': {'lat': -22.74873707502691,
       'lng': -43.10168807024157},
      'sw': {'lat': -23.07659699961192, 'lng': -43.796475958003505}},
     'center': {'lat': -22.90278, 'lng': -43.2075}},
    'highlightedName': '<b>Rio de Janeiro</b>, RJ, Brazil',
    'id': 'geonameid:3451190',
    'longId': '72057594041379126',
    'matchedName': 'Rio de Janeiro, RJ, Brazil',
    'name': 'Rio de Janeiro',
    'slug': 'rio-de-janeiro',
    'woeType': 7},
   'parents': [],
   'what': '',
   'where': 'rio de janeiro'},
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Rest

In [5]:
from pandas.io.json import json_normalize

# assign relevant part of JSON to venues
venues = result['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood,location.crossStreet,venuePage.id
0,54014095498ee8f46cfcc210,Buonasera U.N.O,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1658856841,False,"Av. Rui Barbosa, 218",-22.919765,-43.088614,"[{'label': 'display', 'lat': -22.9197653947608...",BR,Niterói,RJ,Brasil,"[Av. Rui Barbosa, 218, Niterói, RJ]",NaN,NaN,NaN,NaN
1,5a357a127269fe7b957b1f43,Bota,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1658856841,False,Marina da Glória,-22.919681,-43.169387,"[{'label': 'display', 'lat': -22.9196814526607...",BR,Rio de Janeiro,RJ,Brasil,"[Marina da Glória, Rio de Janeiro, RJ, 20021-140]",20021-140,NaN,NaN,NaN
2,4ba50d87f964a5203cd638e3,Spoleto,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1658856841,False,Itaipu Multicenter,-22.940556,-43.056615,"[{'label': 'display', 'lat': -22.9405557954955...",BR,NaN,NaN,Brasil,[Itaipu Multicenter],NaN,NaN,NaN,NaN
3,54d51e37498e64e312ac2eb6,Domino's Pizza & Spoleto,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1658856841,False,R. Savatori,-22.824391,-43.049310,"[{'label': 'display', 'lat': -22.8243905552119...",BR,São Gonçalo,RJ,Brasil,"[R. Savatori, São Gonçalo, RJ]",NaN,NaN,NaN,NaN
4,4d2f38bada7cb60c59fb75d4,Spoleto,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1658856841,False,Boulevard Shopping,-22.820663,-43.046581,"[{'label': 'display', 'lat': -22.8206625455408...",BR,São Gonçalo,RJ,Brasil,"[Boulevard Shopping, São Gonçalo, RJ]",NaN,NaN,NaN,NaN


In [6]:
lojas_rating = []
lojas_price = []
for j in df['id']:
  loja = j
  url = 'https://api.foursquare.com/v2/venues/{}?&cliend_id={}&client_secret={}&oauth_token={}&v={}'.format(loja, ID, SECRET,oauth, vers)
  result2 = requests.get(url).json()
  try:
    rat = result2['response']['venue']['rating']
    lojas_rating.append(rat)
    pri = result2['response']['venue']['price']['tier']
    lojas_price.append(pri)
  except:
    rat = 'N/A'
    lojas_rating.append(rat)
    pri = 'N/A'
    lojas_price.append(pri)


In [7]:
len(lojas_rating)

30

In [8]:
len(df['id'])

30

In [9]:
df['price'] = lojas_price
df['rating'] = lojas_rating

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
df.price

0     2
1     2
2     2
3     1
4     2
5     2
6     2
7     2
8     2
9     2
10    2
11    1
12    1
13    2
14    2
15    3
16    2
17    2
18    3
19    3
20    2
21    2
22    2
23    2
24    2
25    1
26    2
27    3
28    1
29    2
Name: price, dtype: int64

In [12]:
df3 = df[(df.rating != 'N/A') & (df.price != 'N/A')]

In [13]:
df3['fator'] = (df3['rating']*0.7) - (df3['price']*0.3)

## Results
Results are shown on a DataFrame and on a map where the best five places in a quality vs price relation are plotted with a blue circle and everyone else with a red circle.

## Discussion
Since we are using the free version of the API we will get less results than it would be in a premium version. It affects our results, but is easily solved if desired. 



In [14]:
df4 = df3[['name','price','rating','fator','location.address','location.city','location.formattedAddress','location.lat',	'location.lng']]

In [15]:
df4 = df4.sort_values(['fator'], ascending=False).reset_index(drop=True)
df4

,name,price,rating,fator,location.address,location.city,location.formattedAddress,location.lat,location.lng
0,Abbraccio,2,8.8,5.56,Plaza Shopping,Niterói,"[Plaza Shopping (4º Piso), Niterói, RJ, 24020-...",-22.896537,-43.124020
1,Da Carmine,2,8.3,5.21,"R. Mariz e Barros, 305",Niterói,"[R. Mariz e Barros, 305, Niterói, RJ, 24220-121]",-22.905322,-43.103805
2,Don Carlo's Pizza Gourmet,2,8.3,5.21,"R. Francisco Pimentel, 186",Niterói,"[R. Francisco Pimentel, 186, Niterói, RJ, 2421...",-22.904127,-43.122717
3,Da Carmine,3,8.7,5.19,"Estr. Francisco da Cruz Nunes, 2687",Niterói,"[Estr. Francisco da Cruz Nunes, 2687, Niterói,...",-22.948270,-43.030427
4,Torninha,3,8.6,5.12,"R. Nóbrega, 199",Niterói,"[R. Nóbrega, 199, Niterói, RJ, 24220-320]",-22.903959,-43.102770
5,Pizzaria Donatello,1,7.7,5.09,Trevo de Maria Paula,Niterói,"[Trevo de Maria Paula, Niterói, RJ]",-22.879107,-43.026678
6,Domino's Pizza & Spoleto,1,7.7,5.09,R. Savatori,São Gonçalo,"[R. Savatori, São Gonçalo, RJ]",-22.824391,-43.049310
7,D'Amici Ristorante,2,8.1,5.07,"R. Antônio Vieira, 18",Rio de Janeiro,"[R. Antônio Vieira, 18, Rio de Janeiro, RJ, 22...",-22.963617,-43.172881
8,Buonasera U.N.O,2,8.0,5.00,"Av. Rui Barbosa, 218",Niterói,"[Av. Rui Barbosa, 218, Niterói, RJ]",-22.919765,-43.088614
9,Lanchonete Dut,2,7.9,4.93,"R. Gavião Peixoto, 161 - Icaraí",Niterói,"[R. Gavião Peixoto, 161 - Icaraí, Niterói, RJ]",-22.903566,-43.110172


In [16]:
df4.index[9]

9

In [17]:
import folium
from folium import vector_layers

In [18]:
REST = folium.Map(location=[df4['location.lat'][0],	df4['location.lng'][0]], zoom_start=13)

for i in range(0, len(df4['name'])):
  # create a feature group
  re = folium.map.FeatureGroup()

  if i>=4:
    #style the feature group
    re.add_child(folium.CircleMarker([df4['location.lat'][i],df4['location.lng'][i]], color= "red", fill_color="red",radius=6))
  else:
    re.add_child(folium.CircleMarker([df4['location.lat'][i],df4['location.lng'][i]], color = "blue", fill_color="blue",radius=6))
  
  #add the feature to the map
  REST.add_child(re)
  
  #label the marker
  label = '{}º, {}, {}'.format(df4.index[i]+1,df4['name'][i],df4['fator'][i])
  folium.Marker([df4['location.lat'][i],df4['location.lng'][i]], popup=label).add_to(REST)


REST

## Conclusion
That project aims to deliver a solution desired for many people which is find good and affordable places to go to. Which we showed that is possible to be done.